## Iterator and Generator

In [25]:
def meaningless_function(v):
    yield v
    print(f'{v}')

a = meaningless_function('this is meaningless')

In [29]:
next(a)
# there is no more iterable value left

StopIteration: 

In [14]:
# Iterator

class MyIterator:
    
    def __init__(self,xs):
        self.xs = xs
        
    def __next__(self):
        if self.xs:
            return self.xs.pop(0)
        else :
            raise StopIteration

    def __iter__(self):
        return self

c = MyIterator([1,2,3])

In [18]:
next(c)

StopIteration: 

In [49]:
# Generator - don't need __next__, __iter__. 
# the while statement already enables this

def iteration_function(a):
    while True :
        a += 1
        yield a # yield saves the value which was input

In [82]:
# initial value with 2

i = iteration_function(2)

In [ ]:
next(i)
# yield 
# This only happens in the interpreter session

11

In [83]:
for _ in range(0,10):
    val = next(i)

In [84]:
val

12

In [44]:
# yield

def yield_abc():
    yield "A"
    yield "B"
    yield "C"

c = yield_abc()

In [47]:
next(c)

'C'

We can see that the yield_abc becomes a generator. It produces values sequentially whe next() method is called. Once it is looped, it cannot be used again

In [48]:
next(c)

StopIteration: 

In [ ]:
# yield from 

def yield_abc():
    yield from ["A","B","C"]
    
abc = (ch for ch in "ABC")

print(abc) # abc가 제너레이터가 됐다!

for ch in abc:
    print(ch)

<generator object <genexpr> at 0x7f0426f97d60>
A
B
C


In [ ]:
c = yield_abc()

yield from can make a simple generator!

## Coroutine
- yield : used to stop coroutine
- send : send data to the coroutine and execute
- close : end coroutine

In [61]:
def my_talker(words):
    print('this is my name blablabla~')
    
    try :
        while True:
            text = (yield)
            if words in text :
                print(f"This is new text : {words}")
    
    except GeneratorExit:
        print('Ok, this is over now')

In [85]:
c = my_talker('haha')

In [100]:
next(c)

This is new text : haha


In [84]:
c.send('do this one')

This is new text : haha


In [101]:
c.close()

Ok, this is over now


The function becomes an infinite loop

lets make an coroutine based functions

In [104]:
from types import coroutine

@coroutine
def example_function(words):
    print('this is quite silly')
    while True :
        text = (yield) # stop here
        if words in text :
            print(f'this is funny! {text} came out again')
            

In [110]:
a = example_function('silly')

In [142]:
class Example:
    def __init__(self, name):
       self.name = name
      
    def do_something(self):
        print(f'my name is {self.name}') 

@coroutine
def function():
    a = (yield)
    ex1 = Example(a)
    ex1.do_something()

In [136]:
c = ex1.do_something

In [143]:
c = function

In [145]:
c.send('okay')

AttributeError: 'function' object has no attribute 'send'

## Async example
- count 'love' word in pg2600.txt

In [20]:

# main function(Entry point)
def cat(f, case_insensitive, child):
    if case_insensitive :
        line_processor = lambda l: l.lower()
        
    else :
        line_processor = lambda l: l
    
    for line in f:
        child.send(line_processor(line))


def grep(substring, case_insensitive, child):
    if case_insensitive : 
        substring = substring.lower()
    
    while True :
        text = (yield) # yield를 통해서 받아온다. 중간에 넣어지는 값을!
        child.send(text.count(substring))

def count(substring):
    n = 0
    try :
        while True:
            n += (yield)
    
    except GeneratorExit:
        print(substring, n)

In [21]:
cat('pg2600.txt', False, grep('love', False, count('love')))

TypeError: can't send non-None value to a just-started generator

The process just does not start. Why? This is because the function is not a coroutine! we should make it as the coroutine

In [25]:
from types import coroutine

In [28]:
# main function(Entry point)
def cat(f, case_insensitive, child):
    if case_insensitive :
        line_processor = lambda l: l.lower()
        
    else :
        line_processor = lambda l: l
    
    for line in f:
        child.send(line_processor(line))

@coroutine
def grep(substring, case_insensitive, child):
    if case_insensitive : 
        substring = substring.lower()
    
    while True :
        text = (yield) # yield를 통해서 받아온다. 중간에 넣어지는 값을!
        child.send(text.count(substring))

@coroutine
def count(substring):
    n = 0
    try :
        while True:
            n += (yield)
    
    except GeneratorExit:
        print(substring, n)

In [ ]:
if __name__ == "__main__":
    cat('pg2600.txt', False, grep('love', False, count('love')))

TypeError: can't send non-None value to a just-started generator

The code should be started in a .py form. Not in a ipynb form. 